# <a id='toc1_'></a>[Visual Question Answering](#toc0_)

Visual Question Answering (VQA) is the task of answering open-ended questions based on an image. VQA has many applications: Medical VQA, Education purposes, for surveillance and numerous other applications. In this project we will use [VizWiz](https://vizwiz.org/tasks-and-datasets/vqa/) dataset for Visual Question Answering, this dataset was constructed to train models to help visually impaired people.  In the words of creators of VizWiz: “we introduce the visual question answering (VQA) dataset coming from this population, which we call VizWiz-VQA.  It originates from a natural visual question answering setting where blind people each took an image and recorded a spoken question about it, together with 10 crowdsourced answers per visual question.”



<p align="center">
  <img src="Latex Paper\graphics\chapter1\vizwiz_example.png" alt="vizwiz_example" width="500"/>
</p>

- **Note:** This repository is an implementation for [Less is More: Linear Layers on CLIP Features as Powerful VizWiz Model](https://arxiv.org/abs/2206.05281) paper.
- It is really advised to read OpenAI's [CLIP](https://openai.com/blog/clip/) paper before reading this repository if you have enough time.

## <a id='toc1_1_'></a>[Installing Required Libraries](#toc0_)

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import json                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

In [ ]:
df = pd.read_json("Dataset/VQA V2/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json")



In [ ]:
import json
import pandas as pd

# đọc JSON
with open('Dataset/VQA V2/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data["annotations"])

print(df.head())


       question_type multiple_choice_answer  \
0       what is this                    net   
1               what                pitcher   
2  what color is the                 orange   
3            is this                    yes   
4  what color is the                  white   

                                             answers  image_id answer_type  \
0  [{'answer': 'net', 'answer_confidence': 'maybe...    458752       other   
1  [{'answer': 'pitcher', 'answer_confidence': 'y...    458752       other   
2  [{'answer': 'orange', 'answer_confidence': 'ye...    458752       other   
3  [{'answer': 'yes', 'answer_confidence': 'yes',...    458752      yes/no   
4  [{'answer': 'white', 'answer_confidence': 'yes...    262146       other   

   question_id  
0    458752000  
1    458752001  
2    458752002  
3    458752003  
4    262146000  
